# Projet DATA732 - Analyse et Visualisation de données

### SALMA OUKZIZ et EMMA ORSET

Dans ce projet, nous devions analyser un corpus d'articles. 
Dans un premier temps, nous avons décidé d'analyser les mots-clefs du corpus, dans un second temps, nous nous sommes penchés sur les localisations du corpus, et pour finir, nous avons représenté nos données dans un dashboard.

PS : Lorsque vous lancez le projet, n'oubliez pas l'import des librairies et de relancer le kernel à chaque lancement.

In [1]:
import json
import pandas as pd
import plotly.express as px
from dash import dcc, html, Input, Output
from jupyter_dash import JupyterDash

# PARTIE 1 : A quels mots sont liés les 20 mots les plus utilisés ?

Dans cette partie, nous nous interessons aux "keywords" du corpus.

Nous avons dans un premier temps récurépéré les 20 mots-clefs les plus utilisés dans tout le site.
Ensuite, nous avons cherché avec quels autres mots ils sont liés.

Pour ce faire, nous parcourons tous les articles, et à chaque fois qu'un des 20 mots-clefs principal apparaît dans un article, nous ajoutons les autres mots clefs de l'article en tant que "mot lié". A chaque fois qu'un mot lié réaparait dans un autre article avec le mot principal nous incrémentons de 1.

Nous avons pris les 10 mots ayant le plus d'occurences avec le mot principal et nous considérons que ces mots sont très liés au mot principal.

Nous avons affiché cela dans un dashboard où il est possible de choisir un des 20 mot-clef principal pour voir un barchart de ses mots-liés, avec leur nombre d'occurence de relation.

## ETAPE 1 : Chargement des données

In [2]:
file_path = 'analyse_keywords_localisations_data.json'

file = open(file_path, 'r', encoding='utf-8')
data = json.load(file)
file.close()

## ETAPE 2 : Les 20 mots-clefs les plus utilisés globalement

In [3]:
main_kws = dict(sorted(data['metadata-all']['fr']['all']['kws'].items(), key=lambda item: item[1], reverse=True)[:20])

fig = px.bar(pd.DataFrame(main_kws.items(), columns=['Mots-clefs', 'Nombre d\'occurrences']), x='Mots-clefs', y='Nombre d\'occurrences')
fig.show()

## ETAPE 3 : Récupération de tous les mots clefs de chaque article

In [4]:
all_article_kws = []
for y, data_y in data['data-all'].items():
    for m, data_m in data_y.items():
        for d, data_d in data_m.items():
            for a in data_d:
                all_article_kws.append(a['kws'])

## ETAPE 4 : Elaboration du dictionnaire de relation

In [5]:
rel_dict = {kw: {} for kw in main_kws}

def remplir_kw(kw):
    for a in all_article_kws:
        if kw in a:
            for k in a:
                if k in rel_dict[kw] and k != kw:
                    rel_dict[kw][k] += 1
                else:
                    rel_dict[kw][k] = 1

for kw in rel_dict.keys():
    remplir_kw(kw)

for kw, data_kw in rel_dict.items():
    rel_dict[kw] = dict(sorted(data_kw.items(), key=lambda item: item[1], reverse=True)[:10])

list_countries = list(rel_dict.keys())


# PARTIE 2 : De quelles localisations parle-t-on le plus chaque année parmi localisations les plus utilisées globalement ?

Dans cette partie, nous allons étudier les localisations (=pays, villes, lieux, etc.). Nous avons souahité connaitre les 10 localisations les plus utilisées chaque années parmi les 10 localisations les plus utilisées globalement dans le corpus. 

Nous avons récupéré les 10 localisations les plus utilisées globalement, puis nous les avons comparées à chaque années. Si la localisation faisait partie à la fois des 10 localisations les plus utilisées, et aussi présente dans l'année (mais pas forcément très utilisée), alors nous l'ajoutions à notre dictionnaire.

Nous avons mis cela dans un dashboard où l'on peut voir l'évolution année par année.

## ETAPE 1 : Récupérer les 10 localisations les plus utilisées du corpus

In [6]:
main_loc = dict(sorted(data['metadata-all']['fr']['all']['loc'].items(), key=lambda item: item[1], reverse=True)[:10])

fig_loc = px.bar(pd.DataFrame(main_loc.items(), columns=['Localisations', 'Nombre d\'occurrences']), x='Localisations', y='Nombre d\'occurrences')
fig_loc.show()

## ETAPE 2 : Elaboration du dictionnaire des localisations/occurences par année

In [7]:
years = dict(data['metadata-all']['fr']['year'].items()).keys()
dict_loc = {year: {} for year in years}

def remplir_loc(year):
    loc_year = dict(sorted(data['metadata-all']['fr']['year'][year]['loc'].items(), key=lambda item: item[1], reverse=True))
    for loc, data_loc in loc_year.items():
        if loc in main_loc:
            dict_loc[year][loc]=data_loc
    
for year in years:
    remplir_loc(year)

list_data=[]
for y,data in dict_loc.items():
    for loc,occ in data.items():
        list_data.append([y,loc,occ])

#Conversion en dataframe :
df = pd.DataFrame(list_data, columns=['Année', 'Localisation', 'Occurence'])


# PARTIE 3 : Création du dashboard

Nous avons réunis les graphiques obtenus dans la partie 1 et 2 dans un dashboard.

In [8]:
app = JupyterDash(__name__)

app.layout = html.Div(style={"backgroundColor" : "white", "text-align" : "center"}, 
children=[
    html.H1("Analyse de l'article"),

    html.Div(style={"display" : "flex", "flex-wrap" : "wrap", "justify-content" : "center"}, children=[
        html.Div(children=[
            html.H2("Relation entre les mots les plus utilisés"),
            html.Div( children=[dcc.Dropdown(
                                id='dropdown',
                                options=list_countries, 
                                value='mali'),
                                ], style={"width" : "30%", "padding-left" : "30%"}),
            dcc.Graph(id="keywords"), 
            
        ],style={"width" : "600px"}),

        html.Div(children=[
            html.H2("Localisation les plus utilisées par année"),
            dcc.Graph(id="localisations"),
            
        ], style={"width" : "600px"})
    ])
])

@app.callback(
    [Output('keywords', 'figure'), Output('localisations', 'figure')],
    [Input('dropdown', 'value')]
)
def update_output(dropdown):
    localisation = px.scatter(df, x="Localisation", y="Occurence", color="Localisation", size="Occurence", 
            size_max=40, animation_frame="Année", 
            animation_group=df.index, range_y=[0,200], 
            hover_data=df.columns)
    for c in list_countries:
        if dropdown == c:
            keywords = px.bar(pd.DataFrame(rel_dict[c].items(), columns=['Mots-clefs', 'Nombre d\'articles']), x='Mots-clefs', y='Nombre d\'articles', title=f"Les 10 mots les plus liés à '{c}'")
    
    return [keywords, localisation]

app.run_server(mode="inline")